## Dependency Parsing as a Preprocessing Step for Logical Reasoning

In [1]:
# Installs HBOX for Jupyer Notebooks
# !jupyter labextension install @jupyter-widgets/jupyterlab-manager
# !jupyter nbextension enable --py widgetsnbextension
# drive_path = ''

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
drive_path = '/content/drive/MyDrive/FinalProject/'
!pip install transformers

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import gc
import re
import json
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from transformers import RobertaTokenizer
from torch.utils.data import TensorDataset, Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import RobertaConfig, RobertaForSequenceClassification, AdamW

# Enable CUDA Blocking Debugging
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
CUDA_LAUNCH_BLOCKING="1"

# Print GPU Information
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
    print('and then re-execute this cell.')
else:
    print(gpu_info)

Wed Dec 15 21:41:01 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#### Load Data

In [3]:
def load_data(path, test):
    df = pd.DataFrame({'prompt': [], 'label': []})
    data = json.load(open(path))
    for i in range(len(data['context'])):
        # Add BERT tokens to prompt
        prompt = data['context'][str(i)] + ' </s> ' + data['question'][str(i)] + ' </s> '# + data['dep_context'][str(i)] + ' </s> '
        for j in range(4):
            # Add BERT tokens to answer
            answer = data['answers'][str(i)][j]# + ' <d> ' + data['dep_answers'][str(i)][j] + ' <s>'
            
            # Attach 0, 1 label as array
            label = [1] if not test and j == data['label'][str(i)] else [0]
            
            # Append question, answer pair to dataframe
            df = df.append({'prompt': prompt + answer, 'label': [label], 'dep_tree': data['dep_context'][str(i)]}, ignore_index=True)            
    return df

In [4]:
train_data = load_data(drive_path + 'reclor_data_with_dependencies/train.json', False)
val_data = load_data(drive_path + 'reclor_data_with_dependencies/val.json', False)
test_data = load_data(drive_path + 'reclor_data_with_dependencies/test.json', True)

In [5]:
# Describe the token lengths of training data
train_data['prompt'].apply(lambda x: len(re.findall(r'\w+', x))).describe()

count    18552.000000
mean       103.285630
std         23.085243
min         36.000000
25%         88.000000
50%        102.000000
75%        116.000000
max        230.000000
Name: prompt, dtype: float64

#### Initialize tokenizer

In [6]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
tokenizer.add_special_tokens({'additional_special_tokens': ['<d>']})

1

#### Tokenize the data

In [7]:
# Creates a dataloader (which includes an attention mask)
def preprocess(in_, tokenizer, max_len, batch_size, data_class='train'):
    encoded_input = tokenizer(in_['prompt'].values.tolist(), padding=True, max_length=max_len, truncation=True, return_tensors="pt")
    
    if data_class != 'test':
        labels = torch.tensor(in_['label'].values.tolist())
    
    dep_tree = in_['dep_tree'].values.tolist()
    dep_mask = np.array([]) 
    for i in tqdm(range(int(len(dep_tree) / 4))):
        masks = np.array([])
        for j in range(batch_size):
          dep = dep_tree[i + j]
          dependencies = dep.split('],')
          m = np.zeros([768, 768])
          for i in range(len(dependencies)):
              for j in range(len(dependencies)):
                  for x in dependencies[i]:
                      if dependencies[j][0] in x:
                          m[i][j] = 1
                      else:
                          m[i][j] = 0
          np.append(masks, m)
        np.append(dep_mask, masks)
    dep_mask = torch.tensor(dep_mask, dtype=torch.float)
    #print(dep_mask.shape)

    dataset_tensor = TensorDataset(encoded_input['input_ids'], encoded_input['attention_mask'], labels, dep_mask)
    sampler = SequentialSampler(dataset_tensor)
    #sampler = RandomSampler(dataset_tensor) if data_class == "train" else SequentialSampler(dataset_tensor)
    dataloader = DataLoader(dataset_tensor, sampler=sampler, batch_size=batch_size)
    return dataloader

In [ ]:
max_len = 512 # should be 1024
batch_size = 4
train_dataloader = preprocess(train_data, tokenizer, max_len, batch_size)
val_dataloader = preprocess(val_data, tokenizer, max_len, batch_size, data_class="val")
# test_dataloader = preprocess(test_data, tokenizer, max_len, batch_size, data_class="test")

 57%|█████▋    | 2664/4638 [18:33<11:51,  2.77it/s]

#### Train and Evaluate RoBERTa

In [ ]:
def ClearTorch():
    torch.no_grad()
    torch.cuda.empty_cache()
    gc.collect()
    
def Eval(model, dataloader):
    ClearTorch()
    model.eval()
    predictions, true_labels = [], []
    for step, batch in enumerate(tqdm(dataloader)):
        # Call model on batch
        input_ids, attention_mask, labels, dep_mask = batch[0].cuda(), batch[1].cuda(), batch[2].cuda(), batch[3].cuda()
        outputs = model(input_ids, dep_mask=dep_mask, attention_mask=attention_mask, labels=labels)
        
        # Convert output logit to predictions using softmax
        #print(labels)

        predictions.append(torch.nn.functional.softmax(outputs.logits).argmax(0)[1].cpu().numpy().tolist())
        true_labels.append(labels.argmax(0).cpu().numpy().tolist()[0][0])
      
        ClearTorch()

    return float(sum([predictions[i] == true_labels[i] for i in range(len(predictions)) ])) / float(len(predictions))

def Train(model, train_data, lr, n_epoch):
    ClearTorch()
    optimizer = AdamW(model.parameters(), lr=lr)

    for epoch in range(n_epoch):
        print(f"Epoch {epoch}")
        model.train()
        nb_tr_examples, nb_tr_steps, tr_loss = 0, 0, 0

        for step, batch in enumerate(tqdm(train_data)):
            # RoBERTa fine-tuning
            input_ids, attention_mask, labels, dep_mask = batch[0].cuda(), batch[1].cuda(), batch[2].cuda(), batch[3].cuda()
            
            optimizer.zero_grad()
            outputs = model(input_ids, dep_mask = dep_mask, attention_mask=attention_mask, labels=labels)
            outputs.loss.backward()
            optimizer.step()
            
            ClearTorch()
            
            tr_loss += float(outputs.loss)
            nb_tr_steps += 1
            
        print(f"Train loss on epoch {epoch}: {tr_loss / nb_tr_steps}\n")

In [ ]:
ClearTorch()

config = RobertaConfig.from_pretrained('roberta-base')
# config.max_position_embeddings = max_len
model = RobertaForSequenceClassification.from_pretrained('roberta-base', config=config)
model.resize_token_embeddings(len(tokenizer))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if torch.cuda.device_count() > 1:
    model.to(device)
    model = torch.nn.DataParallel(model)
else:
    model.cuda()

In [ ]:
import math
import torch.nn as nn
import torch.nn.functional as F
#from transformers.RobertaForSequenceClassification import RobertaSelfAttention
#print(model.roberta.encoder.layer[0].attention.self)
class RobertaDependencyAttentionMask(type(model.roberta.encoder.layer[0].attention.self)):
    def __init__(self, config):
        super().__init__(config)

    def forward(
        self,
        hidden_states,
        attention_mask=None,
        head_mask=None,
        encoder_hidden_states=None,
        encoder_attention_mask=None,
        past_key_value=None,
        output_attentions=False,
    ):
        mixed_query_layer = self.query(hidden_states)

        # If this is instantiated as a cross-attention module, the keys
        # and values come from an encoder; the attention mask needs to be
        # such that the encoder's padding tokens are not attended to.
        is_cross_attention = encoder_hidden_states is not None

        if is_cross_attention and past_key_value is not None:
            # reuse k,v, cross_attentions
            key_layer = past_key_value[0]
            value_layer = past_key_value[1]
            attention_mask = encoder_attention_mask
        elif is_cross_attention:
            key_layer = self.transpose_for_scores(self.key(encoder_hidden_states))
            value_layer = self.transpose_for_scores(self.value(encoder_hidden_states))
            attention_mask = encoder_attention_mask
        elif past_key_value is not None:
            key_layer = self.transpose_for_scores(self.key(hidden_states))
            value_layer = self.transpose_for_scores(self.value(hidden_states))
            key_layer = torch.cat([past_key_value[0], key_layer], dim=2)
            value_layer = torch.cat([past_key_value[1], value_layer], dim=2)
        else:
            key_layer = self.transpose_for_scores(self.key(hidden_states))
            value_layer = self.transpose_for_scores(self.value(hidden_states))

        #print(mixed_query_layer.shape)
        query_layer = self.transpose_for_scores(mixed_query_layer)
        #print(mixed_query_layer.shape)


        if self.is_decoder:
            # if cross_attention save Tuple(torch.Tensor, torch.Tensor) of all cross attention key/value_states.
            # Further calls to cross_attention layer can then reuse all cross-attention
            # key/value_states (first "if" case)
            # if uni-directional self-attention (decoder) save Tuple(torch.Tensor, torch.Tensor) of
            # all previous decoder key/value_states. Further calls to uni-directional self-attention
            # can concat previous decoder key/value_states to current projected key/value_states (third "elif" case)
            # if encoder bi-directional self-attention `past_key_value` is always `None`
            past_key_value = (key_layer, value_layer)

        # Take the dot product between "query" and "key" to get the raw attention scores.
        # print(query_layer.shape)
        # print(key_layer.shape)
        #print(attention_scores.shape)
        #print(hidden_states.shape)
        attention_scores = torch.matmul(query_layer, key_layer.transpose(-1, -2))
        #print(attention_scores.shape)
        #print(self.dep_mask.shape)
        temp_dep = self.transpose_for_scores(torch.matmul(hidden_states, self.dep_mask))
        #print(temp_dep.shape)

        temp_dep = torch.matmul(temp_dep, temp_dep.transpose(-1, -2))
        # print(temp_dep.shape)
        # print(attention_scores.shape)
        #print(temp_dep.shape)

        attention_scores = torch.mul(attention_scores, temp_dep)

        if self.position_embedding_type == "relative_key" or self.position_embedding_type == "relative_key_query":
            seq_length = hidden_states.size()[1]
            position_ids_l = torch.arange(seq_length, dtype=torch.long, device=hidden_states.device).view(-1, 1)
            position_ids_r = torch.arange(seq_length, dtype=torch.long, device=hidden_states.device).view(1, -1)
            distance = position_ids_l - position_ids_r
            positional_embedding = self.distance_embedding(distance + self.max_position_embeddings - 1)
            positional_embedding = positional_embedding.to(dtype=query_layer.dtype)  # fp16 compatibility

            if self.position_embedding_type == "relative_key":
                relative_position_scores = torch.einsum("bhld,lrd->bhlr", query_layer, positional_embedding)
                attention_scores = attention_scores + relative_position_scores
            elif self.position_embedding_type == "relative_key_query":
                relative_position_scores_query = torch.einsum("bhld,lrd->bhlr", query_layer, positional_embedding)
                relative_position_scores_key = torch.einsum("bhrd,lrd->bhlr", key_layer, positional_embedding)
                attention_scores = attention_scores + relative_position_scores_query + relative_position_scores_key

        attention_scores = attention_scores / math.sqrt(self.attention_head_size)
        if attention_mask is not None:
            # Apply the attention mask is (precomputed for all layers in RobertaModel forward() function)
            attention_scores = attention_scores + attention_mask

        # Normalize the attention scores to probabilities.
        attention_probs = nn.Softmax(dim=-1)(attention_scores)

        # This is actually dropping out entire tokens to attend to, which might
        # seem a bit unusual, but is taken from the original Transformer paper.
        attention_probs = self.dropout(attention_probs)

        # Mask heads if we want to
        if head_mask is not None:
            attention_probs = attention_probs * head_mask

        context_layer = torch.matmul(attention_probs, value_layer)

        context_layer = context_layer.permute(0, 2, 1, 3).contiguous()
        new_context_layer_shape = context_layer.size()[:-2] + (self.all_head_size,)
        context_layer = context_layer.view(*new_context_layer_shape)

        outputs = (context_layer, attention_probs) if output_attentions else (context_layer,)

        if self.is_decoder:
            outputs = outputs + (past_key_value,)
        return outputs

class RobertaDependencyForSequenceClassification(RobertaForSequenceClassification):
    def __init__(self, config):
        super().__init__(config)
        for i, layer in enumerate(self.roberta.encoder.layer):
            attention_state = layer.attention.self.state_dict()
            layer.attention.self = RobertaDependencyAttentionMask(config)
            layer.attention.self.load_state_dict(attention_state)


    def forward(
        self,
        input_ids,
        dep_mask = None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None
    ):
        for i, layer in enumerate(self.roberta.encoder.layer):
            layer.attention.self.dep_mask = dep_mask

        return super().forward(
        input_ids,
        attention_mask=attention_mask,
        token_type_ids=token_type_ids,
        position_ids=position_ids,
        head_mask=head_mask,
        inputs_embeds=inputs_embeds,
        labels=labels,
        output_attentions=output_attentions,
        output_hidden_states=output_hidden_states,
        return_dict=return_dict)

    # def dep_to_attn(self, dep_tree):
    #     masks = []
    #     for dep in dep_tree:
    #       dependencies = dep.split('],')
    #       m = np.zeros([768, 768])
    #       for i in range(len(dependencies)):
    #           for j in range(len(dependencies)):
    #               for x in dependencies[i]:
    #                   if dependencies[j][0] in x:
    #                       m[i][j] = 1
    #                   else:
    #                       m[i][j] = 0
    #       masks.append(m)
    #     return torch.tensor(masks, dtype=torch.float).cuda()



In [ ]:
ClearTorch()

config = RobertaConfig.from_pretrained('roberta-base')
# config.max_position_embeddings = max_len
model = RobertaDependencyForSequenceClassification.from_pretrained('roberta-base', config=config)
model.resize_token_embeddings(len(tokenizer))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if torch.cuda.device_count() > 1:
    model.to(device)
    model = torch.nn.DataParallel(model)
else:
    model.cuda()

In [ ]:
learning_rate = 2e-5
num_epoch = 1

Train(model, train_dataloader, train_trees, learning_rate, num_epoch)

In [ ]:
print(f"Accuracy: {Eval(model, val_dataloader, val_trees)}")